In [ ]:
import sys
!{sys.executable} -m pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 18.6 MB 1.3 MB/s 


In [ ]:
!{sys.executable} -m pip install rectools==0.2.0

In [ ]:
import gdown

url = "https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1W2ckIrN7KKiMZbF00hfH0rFSTKdmPZnr"
gdown.download(url, "tfidf_15.dill")

url = "https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1-CN1i5hKbyHVR9DzbqCGNBn49zNrVZ0K"
gdown.download(url, "cos_15.dill")

url = "https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1-HfvmwMb3oZnfr2G61xKbDjLSRnBhESf"
gdown.download(url, "bm_15.dill")

url = "https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1Sd-Pj94E6vWolj43xDd9xEFpuDf2XCDn"
gdown.download(url, "userknn.py")

Downloading...
From: https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1W2ckIrN7KKiMZbF00hfH0rFSTKdmPZnr
To: /content/tfidf_15.dill
100%|██████████| 313M/313M [00:02<00:00, 151MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1-CN1i5hKbyHVR9DzbqCGNBn49zNrVZ0K
To: /content/cos_15.dill
100%|██████████| 313M/313M [00:02<00:00, 139MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1-HfvmwMb3oZnfr2G61xKbDjLSRnBhESf
To: /content/bm_15.dill
100%|██████████| 313M/313M [00:02<00:00, 156MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1Sd-Pj94E6vWolj43xDd9xEFpuDf2XCDn
To: /content/userknn.py
100%|██████████| 7.17k/7.17k [00:00<00:00, 12.2MB/s]


'userknn.py'

In [ ]:
import warnings
from pprint import pprint

import requests

import numpy as np
import pandas as pd
import dill
from implicit.nearest_neighbours import (
    BM25Recommender,
    CosineRecommender,
    TFIDFRecommender,
)
from rectools import Columns
from rectools.dataset import Dataset
from rectools.models.popular import PopularModel
from scipy.stats import mode
from tqdm.auto import tqdm
from userknn import UserKnn

warnings.filterwarnings("ignore")

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 200)

# Get KION dataset 

<a href="https://ods.ai/competitions/competition-recsys-21/data"> Dataset description [ru] </a>


Download dataset by chunks

In [ ]:
url = "https://storage.yandexcloud.net/itmo-recsys-public-data/kion_train.zip"

req = requests.get(url, stream=True)

with open("kion_train.zip", "wb") as fd:
    total_size_in_bytes = int(req.headers.get("Content-Length", 0))
    progress_bar = tqdm(
        desc="kion dataset download",
        total=total_size_in_bytes,
        unit="iB",
        unit_scale=True,
    )
    for chunk in req.iter_content(chunk_size=2**20):
        progress_bar.update(len(chunk))
        fd.write(chunk)

kion dataset download:   0%|          | 0.00/78.8M [00:00<?, ?iB/s]

In [ ]:
!unzip kion_train.zip

Archive:  kion_train.zip
   creating: kion_train/
  inflating: kion_train/interactions.csv  
  inflating: __MACOSX/kion_train/._interactions.csv  
  inflating: kion_train/users.csv    
  inflating: __MACOSX/kion_train/._users.csv  
  inflating: kion_train/items.csv    
  inflating: __MACOSX/kion_train/._items.csv  


# EDA

In [ ]:
interactions = pd.read_csv("kion_train/interactions.csv")
users = pd.read_csv("kion_train/users.csv")
items = pd.read_csv("kion_train/items.csv")

Rename columns, convert timestamp

In [ ]:
interactions.rename(
    columns={"last_watch_dt": Columns.Datetime, "total_dur": Columns.Weight},
    inplace=True,
)

interactions["datetime"] = pd.to_datetime(interactions["datetime"])

## interactions

In [ ]:
pd.concat([interactions.head(), interactions.tail()])

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0
5476246,648596,12225,2021-08-13,76,0.0
5476247,546862,9673,2021-04-13,2308,49.0
5476248,697262,15297,2021-08-20,18307,63.0
5476249,384202,16197,2021-04-19,6203,100.0
5476250,319709,4436,2021-08-15,3921,45.0


In [ ]:
print(f"Interactions dataframe shape: {interactions.shape}")
print(f"Unique users in interactions: {interactions['user_id'].nunique():_}")
print(f"Unique items in interactions: {interactions['item_id'].nunique():_}")

Interactions dataframe shape: (5476251, 5)
Unique users in interactions: 962_179
Unique items in interactions: 15_706


In [ ]:
max_date = interactions["datetime"].max()
min_date = interactions["datetime"].min()

print(f"min date in interactions: {min_date}")
print(f"max date in interactions: {max_date}")

min date in interactions: 2021-03-13 00:00:00
max date in interactions: 2021-08-22 00:00:00


In [ ]:
interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5476251 entries, 0 to 5476250
Data columns (total 5 columns):
 #   Column       Dtype         
---  ------       -----         
 0   user_id      int64         
 1   item_id      int64         
 2   datetime     datetime64[ns]
 3   weight       int64         
 4   watched_pct  float64       
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 208.9 MB


## users

In [ ]:
pd.concat([users.head(), users.tail()])

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0
840192,339025,age_65_inf,income_0_20,Ж,0
840193,983617,age_18_24,income_20_40,Ж,1
840194,251008,NaN,NaN,NaN,0
840195,590706,NaN,NaN,Ж,0
840196,166555,age_65_inf,income_20_40,Ж,0


In [ ]:
print(f"Users dataframe shape {users.shape}")
print(f"Unique users: {users['user_id'].nunique():_}")

Users dataframe shape (840197, 5)
Unique users: 840_197


## items

In [ ]:
pd.concat([items.head(2), items.tail(2)])

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Грандинетти, Джеральдин Чаплин, Елена Анайя, Каэтано Велозо, Леонор Уотлинг, Лола Дуэньяс, Лолес Леон, Малу Айродо, Мариола Фуэнтес, Пас Вега, Пина Бауш, Ро...",Мелодрама легендарного Педро Альмодовара «Поговори с ней» в 2003 году получила премию «Оскар» за лучший сценарий. Журналист Марко берет интервью у знаменитой женщины-тореро Лидии и вскоре влюбляе...,"Поговори, ней, 2002, Испания, друзья, любовь, сильные, женщины, преодоление, трудностей, отношения, дружба, отношения, паре, отношения, мужчины, женщины, романтические, отношения, потеря, близких,..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон Манцукас, Джон Глейсер, Карл Грин, Кристен Риттер, Лэнс Реддик, Морис Комт, Патрик Кернс, Ребекка Коллинз, Роза Салазар, Росс П. Кук, Стеффи Гроут, Ти...","Уморительная современная комедия на популярную тему о том, как не надо отмечать мальчишник. Главный герой усвоил, что не надо звать на свадьбу своего друга Джейсона, из-за которого он вместо сваде...","Голые, перцы, 2014, США, друзья, свадьбы, преодоление, трудностей, расставания, отношения, дружба, риск, недоразумение, мужская, дружба, мальчишники, девичники"
15961,4538,series,Среди камней,Darklands,2019.0,"драмы, спорт, криминал",Россия,0.0,18.0,NaN,"Марк О’Коннор, Конор МакМахон","Дэйн Уайт О’Хара, Томас Кэйн-Бирн, Джудит Родди, Марк О’Халлоран, Джимми Смоллхорн","Семнадцатилетний Дэмиен мечтает вырваться за пределы своего района и стать профессиональным бойцом. Когда его кумир и старший брат исчезает, парень попадает в чуждый ему мир насилия, наркотиков и ...","Среди, камней, 2019, Россия"
15962,3206,series,Гоша,NaN,2019.0,комедии,Россия,0.0,16.0,NaN,Михаил Миронов,"Мкртыч Арзуманян, Виктория Рунцова","Добродушный Гоша не может выйти из дома, чтобы не попасть в нелепую и курьёзную историю. Но даже неудачники мечтают о любви, и наш герой — не исключение, ведь оптимизма ему не занимать.","Гоша, 2019, Россия"


In [ ]:
print(f"Items dataframe shape {items.shape}")
print(f"Unique item_id: {items['item_id'].nunique():_}")

Items dataframe shape (15963, 14)
Unique item_id: 15_963


#  userkNN model  CV

Compare implicit `CosineRecommender` and `TFIDFRecommender` as an ItemKnn base 



Setting for cv 

In [ ]:
n_folds = 5
unit = "W"
n_units = 1

last_date = interactions[Columns.Datetime].max().normalize()
start_date = last_date - pd.Timedelta(n_folds * n_units + 1, unit=unit)  
print(f"Start date and last date of the test fold: {start_date, last_date}")

Start date and last date of the test fold: (Timestamp('2021-07-11 00:00:00'), Timestamp('2021-08-22 00:00:00'))


### Test fold borders

In [ ]:
from rectools.model_selection import TimeRangeSplit

periods = n_folds + 1
freq = f"{n_units}{unit}"
print(
    f"start_date: {start_date}\n"
    f"last_date: {last_date}\n"
    f"periods: {periods}\n"
    f"freq: {freq}\n"
)

date_range = pd.date_range(
    start=start_date, periods=periods, freq=freq, tz=last_date.tz
)
print(f"Test fold borders: {date_range.values.astype('datetime64[D]')}")

# generator of folds
cv = TimeRangeSplit(
    date_range=date_range,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)
print(f"Real number of folds: {cv.get_n_splits(interactions)}")

start_date: 2021-07-11 00:00:00
last_date: 2021-08-22 00:00:00
periods: 6
freq: 1W

Test fold borders: ['2021-07-11' '2021-07-18' '2021-07-25' '2021-08-01' '2021-08-08'
 '2021-08-15']
Real number of folds: 5


In [ ]:
from rectools.metrics import (
    Precision,
    Recall,
    MeanInvUserFreq,
    Serendipity,
    calc_metrics,
    MAP,
)

# calculate several classic (precision@k and recall@k) and "beyond accuracy" metrics
metrics = {
    "prec@10": Precision(k=10),
    "recall@10": Recall(k=10),
    "novelty": MeanInvUserFreq(k=10),
    "serendipity": Serendipity(k=10),
    "map@10": MAP(k=10),
}

# few simple models to compare
models = {"cosine_itemknn": CosineRecommender()}

# Base model CosineRecommender

In [ ]:
%%time

results = []

fold_iterator = cv.split(interactions, collect_fold_stats=True)

for i_fold, (train_ids, test_ids, fold_info) in enumerate(fold_iterator):
    print(f"\n==================== Fold {i_fold}")
    pprint(fold_info)

    df_train = interactions.iloc[train_ids].copy()
    df_test = interactions.iloc[test_ids][Columns.UserItem].copy()

    catalog = df_train[Columns.Item].unique()
    
    for model_name, model in models.items():
        userknn_model = UserKnn(model=model, N_users=30)
        userknn_model.fit(df_train)
    
        recos = userknn_model.predict(df_test)
    
        metric_values = calc_metrics(
            metrics,
            reco=recos,
            interactions=df_test,
            prev_interactions=df_train,
            catalog=catalog,
        )
    
        fold = {"fold": i_fold, "model": model_name}
        fold.update(metric_values)
        results.append(fold)


==================== Fold 0
{'End date': Timestamp('2021-07-18 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-07-11 00:00:00', freq='W-SUN'),
 'Test': 214489,
 'Test items': 6313,
 'Test users': 84234,
 'Train': 3192875,
 'Train items': 14711,
 'Train users': 640144}


  0%|          | 0/640144 [00:00<?, ?it/s]


==================== Fold 1
{'End date': Timestamp('2021-07-25 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-07-18 00:00:00', freq='W-SUN'),
 'Test': 231207,
 'Test items': 6491,
 'Test users': 87632,
 'Train': 3506106,
 'Train items': 14928,
 'Train users': 687200}


  0%|          | 0/687200 [00:00<?, ?it/s]


==================== Fold 2
{'End date': Timestamp('2021-08-01 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-07-25 00:00:00', freq='W-SUN'),
 'Test': 249396,
 'Test items': 6611,
 'Test users': 93092,
 'Train': 3838180,
 'Train items': 15061,
 'Train users': 734701}


...


==================== Fold 3
{'End date': Timestamp('2021-08-08 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-08-01 00:00:00', freq='W-SUN'),
 'Test': 264039,
 'Test items': 6609,
 'Test users': 98161,
 'Train': 4203885,
 'Train items': 15212,
 'Train users': 788721}


  0%|          | 0/788721 [00:00<?, ?it/s]


==================== Fold 4
{'End date': Timestamp('2021-08-15 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-08-08 00:00:00', freq='W-SUN'),
 'Test': 276699,
 'Test items': 6715,
 'Test users': 101983,
 'Train': 4587708,
 'Train items': 15404,
 'Train users': 842129}


  0%|          | 0/842129 [00:00<?, ?it/s]

CPU times: user 1h 45min 11s, sys: 29.2 s, total: 1h 45min 40s
Wall time: 1h 15min


In [ ]:
df_metrics = pd.DataFrame(results)
df_metrics

,fold,model,prec@10,recall@10,map@10,novelty,serendipity
0,0,cosine_itemknn,0.005023,0.027166,0.005083,7.857216,0.000057
1,1,cosine_itemknn,0.004788,0.024878,0.004595,7.953221,0.000065
2,2,cosine_itemknn,0.004649,0.023626,0.004548,7.989455,0.000071
3,3,cosine_itemknn,0.004650,0.023565,0.004430,8.017822,0.000069
4,4,cosine_itemknn,0.004330,0.021349,0.004101,8.064291,0.000071


In [ ]:
df_metrics.mean()[1:]

prec@10        0.004688
recall@10      0.024117
map@10         0.004552
novelty        7.976401
serendipity    0.000066
dtype: float64

In [ ]:
df_metrics.to_csv("base.csv",index=False)

# Another way to rank

Let's try to rank movies based on the user's favorite genres

In [ ]:
items = items[["item_id", "genres"]]
df_train_with_genres = df_train.merge(items, on="item_id", how="left")

In [ ]:
df_train_with_genres.head()

,user_id,item_id,datetime,weight,watched_pct,genres
0,176549,9506,2021-05-11,4250,72.0,"фэнтези, мультфильм, музыкальные"
1,699317,1659,2021-05-29,8317,100.0,"мультфильм, фэнтези, приключения, комедии"
2,656683,7107,2021-05-09,10,0.0,"драмы, военные, приключения"
3,864613,7638,2021-07-05,14483,100.0,мелодрамы
4,964868,9506,2021-04-30,6725,100.0,"фэнтези, мультфильм, музыкальные"


In [ ]:
df_train_with_genres["genres"] = df_train_with_genres.genres.apply(
    lambda x: x.replace(" ", "").split(",")
)
df_train_with_genres.head()

,user_id,item_id,datetime,weight,watched_pct,genres
0,176549,9506,2021-05-11,4250,72.0,"[фэнтези, мультфильм, музыкальные]"
1,699317,1659,2021-05-29,8317,100.0,"[мультфильм, фэнтези, приключения, комедии]"
2,656683,7107,2021-05-09,10,0.0,"[драмы, военные, приключения]"
3,864613,7638,2021-07-05,14483,100.0,[мелодрамы]
4,964868,9506,2021-04-30,6725,100.0,"[фэнтези, мультфильм, музыкальные]"


In [ ]:
df_train_with_genres = df_train_with_genres.explode("genres")
df_train_with_genres = df_train_with_genres.groupby(
    ["user_id", "genres"]
).item_id.count()
user_genres = df_train_with_genres.reset_index()
user_genres = user_genres.rename({"item_id": "genres_count"}, axis=1)
pd.concat([user_genres.head(), user_genres.tail()])

,user_id,genres,genres_count
0,0,боевики,2
1,0,детективы,2
2,0,драмы,3
3,0,комедии,2
4,0,криминал,1
4713219,1097557,мелодрамы,1
4713220,1097557,мультфильм,1
4713221,1097557,приключения,1
4713222,1097557,семейное,1
4713223,1097557,фантастика,1


In [ ]:
user_genres = user_genres.merge(
    user_genres.groupby("user_id").sum().reset_index(),
    on="user_id",
    how="left",
    suffixes=("", "_sum"),
)
user_genres["weight_genre"] = user_genres.genres_count / user_genres.genres_count_sum
user_genres = user_genres.drop(["genres_count", "genres_count_sum"], axis=1)
pd.concat([user_genres.head(), user_genres.tail()])

,user_id,genres,weight_genre
0,0,боевики,0.133333
1,0,детективы,0.133333
2,0,драмы,0.200000
3,0,комедии,0.133333
4,0,криминал,0.066667
4713219,1097557,мелодрамы,0.125000
4713220,1097557,мультфильм,0.125000
4713221,1097557,приключения,0.125000
4713222,1097557,семейное,0.125000
4713223,1097557,фантастика,0.125000


In [ ]:
def get_user_genres(df_train: pd.DataFrame, items: pd.DataFrame) -> pd.DataFrame:
    """
    The function returns a table with calculated genre weights for the user.
    df_train: dataframe for training the model.
    items: dataframe with movie descriptions.
    """
    df_train_with_genres = df_train.merge(
        items[["item_id", "genres"]], on="item_id", how="left"
    )
    df_train_with_genres["genres"] = df_train_with_genres.genres.apply(
        lambda x: x.replace(" ", "").split(",")
    )
    df_train_with_genres = df_train_with_genres.explode("genres")
    df_train_with_genres = df_train_with_genres.groupby(
        ["user_id", "genres"]
    ).item_id.count()
    user_genres = df_train_with_genres.reset_index()
    user_genres = user_genres.rename({"item_id": "genres_count"}, axis=1)
    user_genres = user_genres.merge(
        user_genres.groupby("user_id").sum().reset_index(),
        on="user_id",
        how="left",
        suffixes=("", "_sum"),
    )
    user_genres["weight_genre"] = (
        user_genres.genres_count / user_genres.genres_count_sum
    )

    return user_genres.drop(["genres_count", "genres_count_sum"], axis=1)

Get a list of movies recommended by the model and add a column with genres to them

In [ ]:
recos = userknn_model.predict(df_test)
recos = recos[["user_id", "item_id"]]
recos = recos.merge(items, on="item_id", how="left")
recos["genres"] = recos.genres.apply(lambda x: x.replace(" ", "").split(","))
recos = recos.explode("genres")
pd.concat([recos.head(), recos.tail()])

,user_id,item_id,genres
0,1097544,15942,биография
0,1097544,15942,документальное
1,1097544,7829,боевики
1,1097544,7829,фантастика
1,1097544,7829,фэнтези
2266312,3,9728,триллеры
2266313,3,10440,триллеры
2266313,3,10440,детективы
2266314,3,15297,драмы
2266314,3,15297,мелодрамы


Add the calculated genre weights to the table

In [ ]:
recos = recos.merge(user_genres, on=["user_id", "genres"], how="left")
recos = recos.fillna(0)
pd.concat([recos.head(), recos.tail()])

,user_id,item_id,genres,weight_genre
0,1097544,15942,биография,0.050000
1,1097544,15942,документальное,0.050000
2,1097544,7829,боевики,0.050000
3,1097544,7829,фантастика,0.050000
4,1097544,7829,фэнтези,0.050000
5764325,3,9728,триллеры,0.236842
5764326,3,10440,триллеры,0.236842
5764327,3,10440,детективы,0.105263
5764328,3,15297,драмы,0.210526
5764329,3,15297,мелодрамы,0.000000


In [ ]:
recos = recos.groupby(["user_id", "item_id"]).mean()
pd.concat([recos.head(), recos.tail()])

weight_genre
user_id item_id              
3       162          0.118421
        1418         0.078947
        1590         0.184211
        2220         0.092105
        2657         0.223684
1097544 4495         0.150000
        7829         0.062500
        9996         0.150000
        15297        0.100000
        15942        0.050000

Sort movies by weight, rank and return N movies

In [ ]:
n = 10
recos = recos.sort_values(["user_id", "weight_genre"], ascending=False)
recos["rank"] = recos.groupby("user_id").cumcount() + 1
recos = recos.reset_index()
recos = recos[recos["rank"] <= n]
pd.concat([recos.head(), recos.tail()])

,user_id,item_id,weight_genre,rank
0,1097544,4495,0.150000,1
1,1097544,9996,0.150000,2
2,1097544,15297,0.100000,3
3,1097544,7829,0.062500,4
4,1097544,4151,0.050000,5
2266303,3,10440,0.171053,6
2266304,3,13849,0.171053,7
2266305,3,9728,0.144737,8
2266306,3,9157,0.138158,9
2266307,3,3734,0.131579,10


In [ ]:
def get_genre_rank(
    recos: pd.DataFrame, items: pd.DataFrame, user_genres: pd.DataFrame, n: int = 10
) -> pd.DataFrame:
    """
    The function ranks the recommendations of the model based on the user's favorite genres.
    recos: dataframe with model recommendations.
    items: dataframe with movie descriptions.
    user_genres: dataframe with genre weights.
    n: number of recommendations for the user.
    """

    recos = recos[["user_id", "item_id"]]
    recos = recos.merge(items[["item_id", "genres"]], on="item_id", how="left")
    recos["genres"] = recos.genres.apply(lambda x: x.replace(" ", "").split(","))
    recos = recos.explode("genres")
    recos = recos.merge(user_genres, on=["user_id", "genres"], how="left")
    recos = recos.fillna(0)
    recos = recos.groupby(["user_id", "item_id"]).mean()
    recos = recos.sort_values(["user_id", "weight_genre"], ascending=False)
    recos["rank"] = recos.groupby("user_id").cumcount() + 1
    recos = recos.reset_index()
    recos = recos[recos["rank"] <= n]

    return recos

In [ ]:
%%time

results = []

fold_iterator = cv.split(interactions, collect_fold_stats=True)

for i_fold, (train_ids, test_ids, fold_info) in enumerate(fold_iterator):
    print(f"\n==================== Fold {i_fold}")
    pprint(fold_info)

    df_train = interactions.iloc[train_ids].copy()
    df_test = interactions.iloc[test_ids][Columns.UserItem].copy()

    catalog = df_train[Columns.Item].unique()
    
    for model_name, model in models.items():
        userknn_model = UserKnn(model=model, N_users=30)
        userknn_model.fit(df_train)

        with open(f"/content/tfidf_cv_{i_fold}.dill", "wb") as f:
            dill.dump(userknn_model, f)        
    
        recos = userknn_model.predict(df_test)

        recos = get_genre_rank(recos, items, user_genres)
    
        metric_values = calc_metrics(
            metrics,
            reco=recos,
            interactions=df_test,
            prev_interactions=df_train,
            catalog=catalog,
        )
    
        fold = {"fold": i_fold, "model": model_name}
        fold.update(metric_values)
        results.append(fold)


==================== Fold 0
{'End date': Timestamp('2021-07-18 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-07-11 00:00:00', freq='W-SUN'),
 'Test': 214489,
 'Test items': 6313,
 'Test users': 84234,
 'Train': 3192875,
 'Train items': 14711,
 'Train users': 640144}


  0%|          | 0/640144 [00:00<?, ?it/s]


==================== Fold 1
{'End date': Timestamp('2021-07-25 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-07-18 00:00:00', freq='W-SUN'),
 'Test': 231207,
 'Test items': 6491,
 'Test users': 87632,
 'Train': 3506106,
 'Train items': 14928,
 'Train users': 687200}


  0%|          | 0/687200 [00:00<?, ?it/s]


==================== Fold 2
{'End date': Timestamp('2021-08-01 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-07-25 00:00:00', freq='W-SUN'),
 'Test': 249396,
 'Test items': 6611,
 'Test users': 93092,
 'Train': 3838180,
 'Train items': 15061,
 'Train users': 734701}


...


==================== Fold 3
{'End date': Timestamp('2021-08-08 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-08-01 00:00:00', freq='W-SUN'),
 'Test': 264039,
 'Test items': 6609,
 'Test users': 98161,
 'Train': 4203885,
 'Train items': 15212,
 'Train users': 788721}


  0%|          | 0/788721 [00:00<?, ?it/s]


==================== Fold 4
{'End date': Timestamp('2021-08-15 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-08-08 00:00:00', freq='W-SUN'),
 'Test': 276699,
 'Test items': 6715,
 'Test users': 101983,
 'Train': 4587708,
 'Train items': 15404,
 'Train users': 842129}


  0%|          | 0/842129 [00:00<?, ?it/s]

CPU times: user 1h 49min 24s, sys: 33.3 s, total: 1h 49min 57s
Wall time: 1h 19min 16s


In [ ]:
df_metrics = pd.DataFrame(results)
df_metrics

,fold,model,prec@10,recall@10,map@10,novelty,serendipity
0,0,cosine_itemknn,0.010486,0.053692,0.018186,6.793281,0.000064
1,1,cosine_itemknn,0.010009,0.049452,0.017317,6.876815,0.000072
2,2,cosine_itemknn,0.009524,0.045695,0.015787,6.887967,0.000080
3,3,cosine_itemknn,0.009618,0.045738,0.015286,6.913778,0.000077
4,4,cosine_itemknn,0.007570,0.035371,0.009519,6.940148,0.000071


In [ ]:
df_metrics.mean()[1:]

prec@10        0.009441
recall@10      0.045990
map@10         0.015219
novelty        6.882398
serendipity    0.000073
dtype: float64

In [ ]:
df_metrics.to_csv("another_rank.csv",index=False)

New way of ranking improved prec@10, map@10 and serendipity

# Pad the recommendations up to N

Let's check how many recommendations we return and supplement them with N popular films

In [ ]:
recos = userknn_model.predict(df_test)

In [ ]:
recos = get_genre_rank(recos, items, user_genres)

In [ ]:
extra_recos = recos.groupby("user_id").agg({"rank": np.max})
extra_recos

,rank
user_id,
3,10
30,10
47,6
53,10
60,10
...,...
1097479,6
1097508,10
1097513,10


It can be seen that there are users who receive less than 10 recommendations

We will select users who lack recommendations and count how many films they lack

In [ ]:
extra_recos = extra_recos.loc[extra_recos["rank"] < 10].reset_index()
extra_recos["add_films"] = 10 - extra_recos["rank"]
extra_recos

,user_id,rank,add_films
0,47,6,4
1,155,6,4
2,158,4,6
3,175,7,3
4,181,7,3
...,...,...,...
27255,1097310,1,9
27256,1097313,8,2
27257,1097403,3,7
27258,1097432,5,5


### Popular Movies

Perhaps a good solution would be to supplement the recommendations with popular films. They can also be recommended to cold users.

We train the model to get a list of popular films

In [ ]:
dataset = Dataset.construct(
    interactions_df=df_train,
    user_features_df=None,
    item_features_df=None,
)

In [ ]:
pop = PopularModel()
pop.fit(dataset)

In [ ]:
# join titles from items
pop.recommend(
    dataset.user_id_map.external_ids[:1],
    dataset=dataset,
    k=10,
    filter_viewed=False,  # True - throw away some items for each user
).merge(items[["item_id", "title"]], on="item_id", how="left")

,user_id,item_id,score,rank,title
0,0,10440,173497.0,1,Хрустальный
1,0,15297,166377.0,2,Клиника счастья
2,0,13865,108030.0,3,Девятаев
3,0,9728,105804.0,4,Гнев человеческий
4,0,4151,79863.0,5,Секреты семейной жизни
5,0,3734,63978.0,6,Прабабушка легкого поведения
6,0,2657,62651.0,7,Подслушано
7,0,4880,51094.0,8,Афера
8,0,142,40254.0,9,Маша
9,0,6809,37536.0,10,Дуров


In [ ]:
popular = (
    pop.recommend(
        dataset.user_id_map.external_ids[:1],
        dataset=dataset,
        k=10,
        filter_viewed=False,  # True - throw away some items for each user
    )
    .merge(items[["item_id", "title"]], on="item_id", how="left")
    .item_id.values
)

In [ ]:
popular

array([10440, 15297, 13865,  9728,  4151,  3734,  2657,  4880,   142,
        6809])

### Supplementing the recommendations

In [ ]:
extra_recos

,user_id,rank,add_films,item_id
0,47,6,4,10440
1,47,6,4,15297
2,47,6,4,13865
3,47,6,4,9728
4,47,6,4,4151
...,...,...,...,...
272595,1097479,6,4,3734
272596,1097479,6,4,2657
272597,1097479,6,4,4880
272598,1097479,6,4,142


Supplement the table with recommendations with popular films, remove popular films that were already on the list and rank

In [ ]:
extra_recos = extra_recos.merge(pd.Series(popular, name="item_id"), how="cross")
extra_recos = extra_recos.loc[
    extra_recos.merge(recos, on=["user_id", "item_id"], how="left").rank_y.isna()
]
extra_recos["rank_pop"] = extra_recos.groupby("user_id").cumcount() + 1
extra_recos

,user_id,rank,add_films,item_id,rank_pop
0,47,6,4,10440,1
4,47,6,4,4151,2
5,47,6,4,3734,3
6,47,6,4,2657,4
7,47,6,4,4880,5
...,...,...,...,...,...
272594,1097479,6,4,4151,3
272596,1097479,6,4,2657,4
272597,1097479,6,4,4880,5
272598,1097479,6,4,142,6


Select the number of films you need and update the rank

In [ ]:
extra_recos = extra_recos.loc[extra_recos.rank_pop <= extra_recos.add_films]
extra_recos["rank"] = extra_recos["rank"] + extra_recos["rank_pop"]
extra_recos = extra_recos.drop(["add_films", "rank_pop"], axis=1)
extra_recos

,user_id,rank,item_id
0,47,7,10440
4,47,8,4151
5,47,9,3734
6,47,10,2657
10,155,7,10440
...,...,...,...
272587,1097432,10,4880
272590,1097479,7,10440
272591,1097479,8,15297
272594,1097479,9,4151


In [ ]:
def get_popular_recos(recos: pd.DataFrame, popular: np.ndarray, n: int = 10):
    """
    The function returns the missing recommendations from the list of popular ones.
    recos: dataframe with model recommendations.
    popular: list of popular movies.
    n: number of recommendations.
    """
    extra_recos = recos.groupby("user_id").agg({"rank": np.max})
    extra_recos = extra_recos.loc[extra_recos["rank"] < n].reset_index()
    extra_recos["add_films"] = n - extra_recos["rank"]

    extra_recos = extra_recos.merge(pd.Series(popular, name="item_id"), how="cross")
    extra_recos = extra_recos.loc[
        extra_recos.merge(recos, on=["user_id", "item_id"], how="left").rank_y.isna()
    ]
    extra_recos["rank_pop"] = extra_recos.groupby("user_id").cumcount() + 1

    extra_recos = extra_recos.loc[extra_recos.rank_pop <= extra_recos.add_films]
    extra_recos["rank"] = extra_recos["rank"] + extra_recos["rank_pop"]

    return extra_recos.drop(["add_films", "rank_pop"], axis=1)

In [ ]:
recos = recos.append(extra_recos)

In [ ]:
recos

,user_id,item_id,weight_genre,rank
0,1097544,9996,0.1500,1
1,1097544,6309,0.1250,2
2,1097544,12768,0.1125,3
3,1097544,10440,0.1000,4
4,1097544,15297,0.1000,5
...,...,...,...,...
272587,1097432,4880,NaN,10
272590,1097479,10440,NaN,7
272591,1097479,15297,NaN,8
272594,1097479,4151,NaN,9


### Checking the quality

Recommendations with popular film 

In [ ]:
%%time

results = []

fold_iterator = cv.split(interactions, collect_fold_stats=True)

for i_fold, (train_ids, test_ids, fold_info) in enumerate(fold_iterator):
    print(f"\n==================== Fold {i_fold}")
    pprint(fold_info)

    df_train = interactions.iloc[train_ids].copy()
    df_test = interactions.iloc[test_ids][Columns.UserItem].copy()

    catalog = df_train[Columns.Item].unique()
    
    for model_name, model in models.items():
        # userknn_model = UserKnn(model=model, N_users=30)
        # userknn_model.fit(df_train)

        with open(f"/content/tfidf_cv_{i_fold}.dill", "rb") as f:
            userknn_model = dill.load(f)        
    
        recos = userknn_model.predict(df_test)

        extra_recos = get_popular_recos(recos, popular)
        recos = recos.append(extra_recos)

    
        metric_values = calc_metrics(
            metrics,
            reco=recos,
            interactions=df_test,
            prev_interactions=df_train,
            catalog=catalog,
        )
    
        fold = {"fold": i_fold, "model": model_name}
        fold.update(metric_values)
        results.append(fold)


==================== Fold 0
{'End date': Timestamp('2021-07-18 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-07-11 00:00:00', freq='W-SUN'),
 'Test': 214489,
 'Test items': 6313,
 'Test users': 84234,
 'Train': 3192875,
 'Train items': 14711,
 'Train users': 640144}

==================== Fold 1
{'End date': Timestamp('2021-07-25 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-07-18 00:00:00', freq='W-SUN'),
 'Test': 231207,
 'Test items': 6491,
 'Test users': 87632,
 'Train': 3506106,
 'Train items': 14928,
 'Train users': 687200}

==================== Fold 2
{'End date': Timestamp('2021-08-01 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-07-25 00:00:00', freq='W-SUN'),
 'Test': 249396,
 'Test items': 6611,
 'Test users': 93092,
 'Train': 3838180,
 'Train items': 15061,
 'Train users': 734701}

==================== Fold 3
{'End date': Timestamp('2021-08-08 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-08-01 00:00:00', freq='W-SUN'),
 'Test': 2

In [ ]:
df_metrics = pd.DataFrame(results)
df_metrics

,fold,model,prec@10,recall@10,map@10,novelty,serendipity
0,0,cosine_itemknn,0.014253,0.079753,0.014496,7.155465,0.000053
1,1,cosine_itemknn,0.013536,0.073183,0.012978,7.210237,0.000061
2,2,cosine_itemknn,0.012462,0.066271,0.012068,7.247984,0.000067
3,3,cosine_itemknn,0.012232,0.064198,0.011849,7.278517,0.000065
4,4,cosine_itemknn,0.011332,0.059110,0.010837,7.332351,0.000067


In [ ]:
df_metrics.mean()[1:]

prec@10        0.012763
recall@10      0.068503
map@10         0.012446
novelty        7.244911
serendipity    0.000063
dtype: float64

In [ ]:
df_metrics.to_csv("add_pop.csv",index=False)

Recommendations with popular film and another rank

In [ ]:
%%time

results = []

fold_iterator = cv.split(interactions, collect_fold_stats=True)

for i_fold, (train_ids, test_ids, fold_info) in enumerate(fold_iterator):
    print(f"\n==================== Fold {i_fold}")
    pprint(fold_info)

    df_train = interactions.iloc[train_ids].copy()
    df_test = interactions.iloc[test_ids][Columns.UserItem].copy()

    catalog = df_train[Columns.Item].unique()
    
    for model_name, model in models.items():
        # userknn_model = UserKnn(model=model, N_users=30)
        # userknn_model.fit(df_train)

        with open(f"/content/tfidf_cv_{i_fold}.dill", "rb") as f:
            userknn_model = dill.load(f)        
    
        recos = userknn_model.predict(df_test)
        recos = get_genre_rank(recos, items, user_genres)

        extra_recos = get_popular_recos(recos, popular)
        recos = recos.append(extra_recos)

    
        metric_values = calc_metrics(
            metrics,
            reco=recos,
            interactions=df_test,
            prev_interactions=df_train,
            catalog=catalog,
        )
    
        fold = {"fold": i_fold, "model": model_name}
        fold.update(metric_values)
        results.append(fold)


==================== Fold 0
{'End date': Timestamp('2021-07-18 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-07-11 00:00:00', freq='W-SUN'),
 'Test': 214489,
 'Test items': 6313,
 'Test users': 84234,
 'Train': 3192875,
 'Train items': 14711,
 'Train users': 640144}

==================== Fold 1
{'End date': Timestamp('2021-07-25 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-07-18 00:00:00', freq='W-SUN'),
 'Test': 231207,
 'Test items': 6491,
 'Test users': 87632,
 'Train': 3506106,
 'Train items': 14928,
 'Train users': 687200}

==================== Fold 2
{'End date': Timestamp('2021-08-01 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-07-25 00:00:00', freq='W-SUN'),
 'Test': 249396,
 'Test items': 6611,
 'Test users': 93092,
 'Train': 3838180,
 'Train items': 15061,
 'Train users': 734701}

==================== Fold 3
{'End date': Timestamp('2021-08-08 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-08-01 00:00:00', freq='W-SUN'),
 'Test': 2

In [ ]:
df_metrics = pd.DataFrame(results)
df_metrics

,fold,model,prec@10,recall@10,map@10,novelty,serendipity
0,0,cosine_itemknn,0.019717,0.106279,0.027599,6.091531,0.000060
1,1,cosine_itemknn,0.018757,0.097758,0.025699,6.133831,0.000068
2,2,cosine_itemknn,0.017337,0.088340,0.023307,6.146496,0.000076
3,3,cosine_itemknn,0.017200,0.086371,0.022706,6.174474,0.000073
4,4,cosine_itemknn,0.014572,0.073132,0.016255,6.208208,0.000067


In [ ]:
df_metrics.mean()[1:]

prec@10        0.017516
recall@10      0.090376
map@10         0.023113
novelty        6.150908
serendipity    0.000069
dtype: float64

In [ ]:
df_metrics.to_csv("add_rank_and_pop.csv",index=False)

Compare all previous options

In [ ]:
pd.DataFrame(
    {
        "CosineRecommender": [0.004688, 0.024117, 0.004552, 7.976401, 0.000066],
        "model_with_popular": [0.012763, 0.068503, 0.012446, 7.244911, 0.000063],
        "model_genres_rank": [0.009441, 0.045990, 0.015219, 6.882398, 0.000073],
        "model_genres_rank_with_popular": [
            0.017516,
            0.090376,
            0.023113,
            6.150908,
            0.000069,
        ],
    },
    index=["prec@10", "recall@10", "map@10", "novelty", "serendipity"],
)

,CosineRecommender,model_with_popular,model_genres_rank,model_genres_rank_with_popular
prec@10,0.004688,0.012763,0.009441,0.017516
recall@10,0.024117,0.068503,0.045990,0.090376
map@10,0.004552,0.012446,0.015219,0.023113
novelty,7.976401,7.244911,6.882398,6.150908
serendipity,0.000066,0.000063,0.000073,0.000069


Adding popular movies improved classification and ranking metrics but reduced novelty.

Rank movies based on the user's favorite genres improved precision and serendipity but also reduced novelty.

As a result, our assumptions greatly improved classification and ranking metrics, had almost no effect on serendipity and greatly reduced novelty

# Different types of kNN models

Setting for cv 

In [ ]:
n_folds = 5
unit = "W"
n_units = 1

last_date = interactions[Columns.Datetime].max().normalize()
start_date = last_date - pd.Timedelta(n_folds * n_units + 1, unit=unit)  
print(f"Start date and last date of the test fold: {start_date, last_date}")

Start date and last date of the test fold: (Timestamp('2021-07-11 00:00:00'), Timestamp('2021-08-22 00:00:00'))


In [ ]:
from rectools.metrics import (
    Precision,
    Recall,
    MeanInvUserFreq,
    Serendipity,
    calc_metrics,
    MAP,
)

# calculate several classic (precision@k and recall@k) and "beyond accuracy" metrics
metrics = {
    "prec@10": Precision(k=10),
    "recall@10": Recall(k=10),
    "novelty": MeanInvUserFreq(k=10),
    "serendipity": Serendipity(k=10),
    "map@10": MAP(k=10),
}

# few simple models to compare
models = {
    "cosine_itemknn": CosineRecommender(),
    "tfidf_itemknn": TFIDFRecommender(),
    "bm25_itemknn": BM25Recommender(),
}

In [ ]:
%%time
results = []

fold_iterator = cv.split(interactions, collect_fold_stats=True)

for i_fold, (train_ids, test_ids, fold_info) in enumerate(fold_iterator):
    print(f"\n==================== Fold {i_fold}")
    pprint(fold_info)

    df_train = interactions.iloc[train_ids].copy()
    df_test = interactions.iloc[test_ids][Columns.UserItem].copy()

    catalog = df_train[Columns.Item].unique()
    
    for model_name, model in models.items():
        userknn_model = UserKnn(model=model, N_users=15)
        userknn_model.fit(df_train)
   
        recos = userknn_model.predict(df_test)
        recos = get_genre_rank(recos, items, user_genres)
        extra_recos = get_popular_recos(recos, popular)
        recos = recos.append(extra_recos)
       
        metric_values = calc_metrics(
            metrics,
            reco=recos,
            interactions=df_test,
            prev_interactions=df_train,
            catalog=catalog,
        )
    
        fold = {"fold": i_fold, "model": model_name}
        fold.update(metric_values)
        results.append(fold)


==================== Fold 0
{'End date': Timestamp('2021-07-18 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-07-11 00:00:00', freq='W-SUN'),
 'Test': 214489,
 'Test items': 6313,
 'Test users': 84234,
 'Train': 3192875,
 'Train items': 14711,
 'Train users': 640144}


  0%|          | 0/640144 [00:00<?, ?it/s]

  0%|          | 0/640144 [00:00<?, ?it/s]

  0%|          | 0/640144 [00:00<?, ?it/s]


==================== Fold 1
{'End date': Timestamp('2021-07-25 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-07-18 00:00:00', freq='W-SUN'),
 'Test': 231207,
 'Test items': 6491,
 'Test users': 87632,
 'Train': 3506106,
 'Train items': 14928,
 'Train users': 687200}


...


==================== Fold 4
{'End date': Timestamp('2021-08-15 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-08-08 00:00:00', freq='W-SUN'),
 'Test': 276699,
 'Test items': 6715,
 'Test users': 101983,
 'Train': 4587708,
 'Train items': 15404,
 'Train users': 842129}


  0%|          | 0/842129 [00:00<?, ?it/s]

  0%|          | 0/842129 [00:00<?, ?it/s]

  0%|          | 0/842129 [00:00<?, ?it/s]

CPU times: user 8h 42min 58s, sys: 4min 23s, total: 8h 47min 21s
Wall time: 6h 16min 52s


In [ ]:
df_metrics = pd.DataFrame(results)

In [ ]:
df_metrics

,fold,model,prec@10,recall@10,map@10,novelty,serendipity
0,0,cosine_itemknn,0.018196,0.092796,0.020320,5.898961,0.000050
1,0,tfidf_itemknn,0.019890,0.095571,0.022668,6.178908,0.000053
2,0,bm25_itemknn,0.013541,0.068287,0.016346,7.492785,0.000098
3,1,cosine_itemknn,0.017102,0.085029,0.018562,5.933398,0.000056
4,1,tfidf_itemknn,0.018565,0.087524,0.020928,6.214025,0.000064
5,1,bm25_itemknn,0.013035,0.063445,0.015384,7.555709,0.000115
6,2,cosine_itemknn,0.015900,0.077462,0.017083,5.948926,0.000061
7,2,tfidf_itemknn,0.017470,0.080263,0.019632,6.229384,0.000065
8,2,bm25_itemknn,0.012437,0.059548,0.014669,7.574779,0.000119
9,3,cosine_itemknn,0.015840,0.075853,0.016969,5.972172,0.000060


In [ ]:
df_metrics.groupby("model").mean()[df_metrics.columns[3:]]

,prec@10,recall@10,map@10,novelty,serendipity
model,,,,,
bm25_itemknn,0.012519,0.060398,0.014790,7.562549,0.000115
cosine_itemknn,0.016361,0.080449,0.017722,5.951118,0.000058
tfidf_itemknn,0.017937,0.083055,0.020029,6.222268,0.000063


It can be seen that TFIDFRecommender shows the best result for all metrics

Let's try a different number of neighbors for the best model

In [ ]:
from rectools.metrics import (
    Precision,
    Recall,
    MeanInvUserFreq,
    Serendipity,
    calc_metrics,
    MAP,
)

# calculate several classic (precision@k and recall@k) and "beyond accuracy" metrics
metrics = {
    "prec@10": Precision(k=10),
    "recall@10": Recall(k=10),
    "novelty": MeanInvUserFreq(k=10),
    "serendipity": Serendipity(k=10),
    "map@10": MAP(k=10),
}

# few simple models to compare
models = {
    "tfidf_itemknn": TFIDFRecommender(),
}

In [ ]:
%%time
results = []

fold_iterator = cv.split(interactions, collect_fold_stats=True)

for i_fold, (train_ids, test_ids, fold_info) in enumerate(fold_iterator):
    print(f"\n==================== Fold {i_fold}")
    pprint(fold_info)

    df_train = interactions.iloc[train_ids].copy()
    df_test = interactions.iloc[test_ids][Columns.UserItem].copy()

    catalog = df_train[Columns.Item].unique()
    
    for model_name, model in models.items():
        userknn_model = UserKnn(model=model, N_users=30)
        userknn_model.fit(df_train)
   
        recos = userknn_model.predict(df_test)
        recos = get_genre_rank(recos, items, user_genres)
        extra_recos = get_popular_recos(recos, popular)
        recos = recos.append(extra_recos)
        
        metric_values = calc_metrics(
            metrics,
            reco=recos,
            interactions=df_test,
            prev_interactions=df_train,
            catalog=catalog,
        )
    
        fold = {"fold": i_fold, "model": model_name}
        fold.update(metric_values)
        results.append(fold)


==================== Fold 0
{'End date': Timestamp('2021-07-18 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-07-11 00:00:00', freq='W-SUN'),
 'Test': 214489,
 'Test items': 6313,
 'Test users': 84234,
 'Train': 3192875,
 'Train items': 14711,
 'Train users': 640144}


  0%|          | 0/640144 [00:00<?, ?it/s]


==================== Fold 1
{'End date': Timestamp('2021-07-25 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-07-18 00:00:00', freq='W-SUN'),
 'Test': 231207,
 'Test items': 6491,
 'Test users': 87632,
 'Train': 3506106,
 'Train items': 14928,
 'Train users': 687200}


  0%|          | 0/687200 [00:00<?, ?it/s]


==================== Fold 2
{'End date': Timestamp('2021-08-01 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-07-25 00:00:00', freq='W-SUN'),
 'Test': 249396,
 'Test items': 6611,
 'Test users': 93092,
 'Train': 3838180,
 'Train items': 15061,
 'Train users': 734701}


...


==================== Fold 3
{'End date': Timestamp('2021-08-08 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-08-01 00:00:00', freq='W-SUN'),
 'Test': 264039,
 'Test items': 6609,
 'Test users': 98161,
 'Train': 4203885,
 'Train items': 15212,
 'Train users': 788721}


  0%|          | 0/788721 [00:00<?, ?it/s]


==================== Fold 4
{'End date': Timestamp('2021-08-15 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-08-08 00:00:00', freq='W-SUN'),
 'Test': 276699,
 'Test items': 6715,
 'Test users': 101983,
 'Train': 4587708,
 'Train items': 15404,
 'Train users': 842129}


  0%|          | 0/842129 [00:00<?, ?it/s]

CPU times: user 2h 17min 39s, sys: 1min 2s, total: 2h 18min 41s
Wall time: 1h 40min 18s


In [ ]:
df_metrics = pd.DataFrame(results)
df_metrics
df_metrics.groupby("model").mean()[df_metrics.columns[2:]]

,prec@10,recall@10,map@10,novelty,serendipity
model,,,,,
tfidf_itemknn,0.013602,0.065017,0.015142,7.52958,0.000052


In [ ]:
%%time
results = []

fold_iterator = cv.split(interactions, collect_fold_stats=True)

for i_fold, (train_ids, test_ids, fold_info) in enumerate(fold_iterator):
    print(f"\n==================== Fold {i_fold}")
    pprint(fold_info)

    df_train = interactions.iloc[train_ids].copy()
    df_test = interactions.iloc[test_ids][Columns.UserItem].copy()

    catalog = df_train[Columns.Item].unique()
    
    for model_name, model in models.items():
        userknn_model = UserKnn(model=model, N_users=50)
        userknn_model.fit(df_train)
   
        recos = userknn_model.predict(df_test)
        recos = get_genre_rank(recos, items, user_genres)
        extra_recos = get_popular_recos(recos, popular)
        recos = recos.append(extra_recos)
        
        metric_values = calc_metrics(
            metrics,
            reco=recos,
            interactions=df_test,
            prev_interactions=df_train,
            catalog=catalog,
        )
    
        fold = {"fold": i_fold, "model": model_name}
        fold.update(metric_values)
        results.append(fold)
        


==================== Fold 0
{'End date': Timestamp('2021-07-18 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-07-11 00:00:00', freq='W-SUN'),
 'Test': 214489,
 'Test items': 6313,
 'Test users': 84234,
 'Train': 3192875,
 'Train items': 14711,
 'Train users': 640144}


  0%|          | 0/640144 [00:00<?, ?it/s]


==================== Fold 1
{'End date': Timestamp('2021-07-25 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-07-18 00:00:00', freq='W-SUN'),
 'Test': 231207,
 'Test items': 6491,
 'Test users': 87632,
 'Train': 3506106,
 'Train items': 14928,
 'Train users': 687200}


  0%|          | 0/687200 [00:00<?, ?it/s]


==================== Fold 2
{'End date': Timestamp('2021-08-01 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-07-25 00:00:00', freq='W-SUN'),
 'Test': 249396,
 'Test items': 6611,
 'Test users': 93092,
 'Train': 3838180,
 'Train items': 15061,
 'Train users': 734701}


...


==================== Fold 3
{'End date': Timestamp('2021-08-08 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-08-01 00:00:00', freq='W-SUN'),
 'Test': 264039,
 'Test items': 6609,
 'Test users': 98161,
 'Train': 4203885,
 'Train items': 15212,
 'Train users': 788721}


  0%|          | 0/788721 [00:00<?, ?it/s]


==================== Fold 4
{'End date': Timestamp('2021-08-15 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-08-08 00:00:00', freq='W-SUN'),
 'Test': 276699,
 'Test items': 6715,
 'Test users': 101983,
 'Train': 4587708,
 'Train items': 15404,
 'Train users': 842129}


  0%|          | 0/842129 [00:00<?, ?it/s]

CPU times: user 2h 27min 32s, sys: 1min 6s, total: 2h 28min 39s
Wall time: 1h 44min 59s


In [ ]:
df_metrics = pd.DataFrame(results)
df_metrics
df_metrics.groupby("model").mean()[df_metrics.columns[2:]]

,prec@10,recall@10,map@10,novelty,serendipity
model,,,,,
tfidf_itemknn,0.013602,0.065017,0.015142,7.52958,0.000052


In [ ]:
pd.DataFrame(
    {
        "TfidfRecommender_15": [0.017937, 0.083055, 0.020029, 6.222268, 0.000063],
        "TfidfRecommender_30": [0.013602, 0.065017, 0.015142, 7.52958, 0.000052],
        "TfidfRecommender_50": [0.013602, 0.065017, 0.015142, 7.52958, 0.000052],
    },
    index=["prec@10", "recall@10", "map@10", "novelty", "serendipity"],
)

,TfidfRecommender_15,TfidfRecommender_30,TfidfRecommender_50
prec@10,0.017937,0.013602,0.013602
recall@10,0.083055,0.065017,0.065017
map@10,0.020029,0.015142,0.015142
novelty,6.222268,7.529580,7.529580
serendipity,0.000063,0.000052,0.000052


# Model blending

In [ ]:
# setting for cv 
n_folds = 1
unit = "W"
n_units = 1

last_date = interactions[Columns.Datetime].max().normalize()
start_date = last_date - pd.Timedelta(n_folds * n_units + 1, unit=unit)  
print(f"Start date and last date of the test fold: {start_date, last_date}")

Start date and last date of the test fold: (Timestamp('2021-08-08 00:00:00'), Timestamp('2021-08-22 00:00:00'))


In [ ]:
from rectools.model_selection import TimeRangeSplit

periods = n_folds + 1
freq = f"{n_units}{unit}"
print(
    f"start_date: {start_date}\n"
    f"last_date: {last_date}\n"
    f"periods: {periods}\n"
    f"freq: {freq}\n"
)

date_range = pd.date_range(
    start=start_date, periods=periods, freq=freq, tz=last_date.tz
)
print(f"Test fold borders: {date_range.values.astype('datetime64[D]')}")

# generator of folds
cv = TimeRangeSplit(
    date_range=date_range,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)
print(f"Real number of folds: {cv.get_n_splits(interactions)}")

start_date: 2021-08-08 00:00:00
last_date: 2021-08-22 00:00:00
periods: 2
freq: 1W

Test fold borders: ['2021-08-08' '2021-08-15']
Real number of folds: 1


In [ ]:
fold_iterator = cv.split(interactions, collect_fold_stats=True)

for i_fold, (train_ids, test_ids, fold_info) in enumerate(fold_iterator):
    print(f"\n==================== Fold {i_fold}")
    pprint(fold_info)

    df_train = interactions.iloc[train_ids].copy()
    df_test = interactions.iloc[test_ids][Columns.UserItem].copy()

    catalog = df_train[Columns.Item].unique()



==================== Fold 0
{'End date': Timestamp('2021-08-15 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-08-08 00:00:00', freq='W-SUN'),
 'Test': 276699,
 'Test items': 6715,
 'Test users': 101983,
 'Train': 4587708,
 'Train items': 15404,
 'Train users': 842129}


In [ ]:
def get_user_genres(df_train: pd.DataFrame, items: pd.DataFrame) -> pd.DataFrame:
    """
    The function returns a table with calculated genre weights for the user.
    df_train: dataframe for training the model.
    items: dataframe with movie descriptions.
    """
    df_train_with_genres = df_train.merge(
        items[["item_id", "genres"]], on="item_id", how="left"
    )
    df_train_with_genres["genres"] = df_train_with_genres.genres.apply(
        lambda x: x.replace(" ", "").split(",")
    )
    df_train_with_genres = df_train_with_genres.explode("genres")
    df_train_with_genres = df_train_with_genres.groupby(
        ["user_id", "genres"]
    ).item_id.count()
    user_genres = df_train_with_genres.reset_index()
    user_genres = user_genres.rename({"item_id": "genres_count"}, axis=1)
    user_genres = user_genres.merge(
        user_genres.groupby("user_id").sum().reset_index(),
        on="user_id",
        how="left",
        suffixes=("", "_sum"),
    )
    user_genres["weight_genre"] = (
        user_genres.genres_count / user_genres.genres_count_sum
    )

    return user_genres.drop(["genres_count", "genres_count_sum"], axis=1)

In [ ]:
user_genres = get_user_genres(df_train, items)

Load trained models

In [ ]:
import dill

with open("/content/tfidf_15.dill", "rb") as f:
    tfidf_itemknn = dill.load(f)

with open("/content/cos_15.dill", "rb") as f:
    cosine_itemknn = dill.load(f)

with open("/content/bm_15.dill", "rb") as f:
    bm25_itemknn = dill.load(f)

In [ ]:
def get_genre_rank(
    recos: pd.DataFrame, items: pd.DataFrame, user_genres: pd.DataFrame, n: int = 10
) -> pd.DataFrame:
    """
    The function ranks the recommendations of the model based on the user's favorite genres.
    recos: dataframe with model recommendations.
    items: dataframe with movie descriptions.
    user_genres: dataframe with genre weights.
    n: number of recommendations for the user.
    """

    recos = recos[["user_id", "item_id"]]
    recos = recos.merge(items[["item_id", "genres"]], on="item_id", how="left")
    recos["genres"] = recos.genres.apply(lambda x: x.replace(" ", "").split(","))
    recos = recos.explode("genres")
    recos = recos.merge(user_genres, on=["user_id", "genres"], how="left")
    recos = recos.fillna(0)
    recos = recos.groupby(["user_id", "item_id"]).mean()
    recos = recos.sort_values(["user_id", "weight_genre"], ascending=False)
    recos["rank"] = recos.groupby("user_id").cumcount() + 1
    recos = recos.reset_index()
    recos = recos[recos["rank"] <= n]

    return recos

Let's make predictions for the test sample and change the ranking

In [ ]:
recos_cos = cosine_itemknn.predict(df_test)
recos_cos = get_genre_rank(recos_cos, items, user_genres)

In [ ]:
recos_tfidf = tfidf_itemknn.predict(df_test)
recos_tfidf = get_genre_rank(recos_tfidf, items, user_genres)

In [ ]:
recos_bm25 = bm25_itemknn.predict(df_test)
recos_bm25 = get_genre_rank(recos_bm25, items, user_genres)

Let's combine all the predictions, sort the values by weights, remove duplicates and re-rank them

In [ ]:
recos = pd.concat([recos_cos, recos_tfidf, recos_bm25])
recos = recos.sort_values(by=["user_id", "weight_genre"], ascending=False)
recos = recos.drop_duplicates(["user_id", "item_id"], keep="first")
recos["rank"] = recos.groupby("user_id").cumcount() + 1

Calculate how many recommendations we return for each user

In [ ]:
extra_recos = recos.groupby("user_id").agg({"rank": np.max})

We will select users who lack recommendations and count how many films they lack

In [ ]:
extra_recos = extra_recos.loc[extra_recos["rank"] < 10].reset_index()
extra_recos["add_films"] = 10 - extra_recos["rank"]
extra_recos

,user_id,rank,add_films
0,47,8,2
1,158,4,6
2,276,1,9
3,348,1,9
4,448,4,6
...,...,...,...
14415,1097119,3,7
14416,1097146,5,5
14417,1097161,7,3
14418,1097403,6,4


Complement the list of recommendations with popular films

In [ ]:
extra_recos = get_popular_recos(recos, popular)

In [ ]:
recos = recos.append(extra_recos)

In [ ]:
recos

,user_id,item_id,weight_genre,rank
0,1097544,9996,0.150,1
1,1097544,6309,0.125,2
0,1097544,4811,0.125,3
1,1097544,5045,0.125,4
2,1097544,5873,0.125,5
...,...,...,...,...
144182,1097403,13865,NaN,7
144183,1097403,9728,NaN,8
144184,1097403,4151,NaN,9
144185,1097403,3734,NaN,10


Let's calculate the metrics

In [ ]:
from rectools.metrics import (
    Precision,
    Recall,
    MeanInvUserFreq,
    Serendipity,
    calc_metrics,
    MAP,
)

# calculate several classic (precision@k and recall@k) and "beyond accuracy" metrics
metrics = {
    "prec@10": Precision(k=10),
    "recall@10": Recall(k=10),
    "novelty": MeanInvUserFreq(k=10),
    "serendipity": Serendipity(k=10),
    "map@10": MAP(k=10),
}


In [ ]:
metric_values = calc_metrics(
    metrics,
    reco=recos,
    interactions=df_test,
    prev_interactions=df_train,
    catalog=catalog,
)

metric_values

{'prec@10': 0.0125216947922693,
 'recall@10': 0.06189741141988651,
 'map@10': 0.01551940264689753,
 'novelty': 7.169950526215326,
 'serendipity': 0.00011158330777107625}

Checking the quality on cv

In [ ]:
from rectools.metrics import (
    Precision,
    Recall,
    MeanInvUserFreq,
    Serendipity,
    calc_metrics,
    MAP,
)

# calculate several classic (precision@k and recall@k) and "beyond accuracy" metrics
metrics = {
    "prec@10": Precision(k=10),
    "recall@10": Recall(k=10),
    "novelty": MeanInvUserFreq(k=10),
    "serendipity": Serendipity(k=10),
    "map@10": MAP(k=10),
}

# few simple models to compare
models = {
    "cosine_itemknn": CosineRecommender(),
    "tfidf_itemknn": TFIDFRecommender(),
    "bm25_itemknn": BM25Recommender(),
}

In [ ]:
# setting for cv 
n_folds = 5
unit = "W"
n_units = 1

last_date = interactions[Columns.Datetime].max().normalize()
start_date = last_date - pd.Timedelta(n_folds * n_units + 1, unit=unit)  
print(f"Start date and last date of the test fold: {start_date, last_date}")

Start date and last date of the test fold: (Timestamp('2021-08-08 00:00:00'), Timestamp('2021-08-22 00:00:00'))


In [ ]:
from rectools.model_selection import TimeRangeSplit

periods = n_folds + 1
freq = f"{n_units}{unit}"
print(
    f"start_date: {start_date}\n"
    f"last_date: {last_date}\n"
    f"periods: {periods}\n"
    f"freq: {freq}\n"
)

date_range = pd.date_range(
    start=start_date, periods=periods, freq=freq, tz=last_date.tz
)
print(f"Test fold borders: {date_range.values.astype('datetime64[D]')}")

# generator of folds
cv = TimeRangeSplit(
    date_range=date_range,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)
print(f"Real number of folds: {cv.get_n_splits(interactions)}")

start_date: 2021-08-08 00:00:00
last_date: 2021-08-22 00:00:00
periods: 2
freq: 1W

Test fold borders: ['2021-08-08' '2021-08-15']
Real number of folds: 1


In [ ]:
%%time
results = []

fold_iterator = cv.split(interactions, collect_fold_stats=True)

for i_fold, (train_ids, test_ids, fold_info) in enumerate(fold_iterator):
    print(f"\n==================== Fold {i_fold}")
    pprint(fold_info)

    df_train = interactions.iloc[train_ids].copy()
    df_test = interactions.iloc[test_ids][Columns.UserItem].copy()

    catalog = df_train[Columns.Item].unique()

    userknn_cosine = UserKnn(model=CosineRecommender(), N_users=30)
    userknn_cosine.fit(df_train)   
    recos_cosine = userknn_cosine.predict(df_test)        
    recos_cosine = get_genre_rank(recos_cosine, items, user_genres)

    userknn_tfidf = UserKnn(model=TFIDFRecommender(), N_users=30)
    userknn_tfidf.fit(df_train)   
    recos_tfidf = userknn_tfidf.predict(df_test)        
    recos_tfidf = get_genre_rank(recos_tfidf, items, user_genres)

    userknn_bm25 = UserKnn(model=BM25Recommender(), N_users=30)
    userknn_bm25.fit(df_train)   
    recos_bm25 = userknn_bm25.predict(df_test)        
    recos_bm25 = get_genre_rank(recos_bm25, items, user_genres)

    recos = pd.concat([recos_cosine, recos_tfidf, recos_bm25])
    recos = recos.sort_values(by=["user_id", "weight_genre"], ascending=False)
    recos = recos.drop_duplicates(["user_id", "item_id"], keep="first")
    recos["rank"] = recos.groupby("user_id").cumcount() + 1 
    recos = recos.loc[recos["rank"] <= 10]

    extra_recos = get_popular_recos(recos, popular)
    recos = recos.append(extra_recos)
       
    metric_values = calc_metrics(
            metrics,
            reco=recos,
            interactions=df_test,
            prev_interactions=df_train,
            catalog=catalog,
        )
    print(metric_values)
    
    fold = {"fold": i_fold, "model": "blending"}
    fold.update(metric_values)
    results.append(fold)


==================== Fold 0
{'End date': Timestamp('2021-07-25 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-07-18 00:00:00', freq='W-SUN'),
 'Test': 231207,
 'Test items': 6491,
 'Test users': 87632,
 'Train': 3506106,
 'Train items': 14928,
 'Train users': 687200}


  0%|          | 0/687200 [00:00<?, ?it/s]

  0%|          | 0/687200 [00:00<?, ?it/s]

  0%|          | 0/687200 [00:00<?, ?it/s]

{'prec@10': 0.0173897662954172, 'recall@10': 0.08785209775497947, 'map@10': 0.027736285607939296, 'novelty': 7.283353827592802, 'serendipity': 0.00013563620249392434}

==================== Fold 1
{'End date': Timestamp('2021-08-01 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-07-25 00:00:00', freq='W-SUN'),
 'Test': 249396,
 'Test items': 6611,
 'Test users': 93092,
 'Train': 3838180,
 'Train items': 15061,
 'Train users': 734701}


...

  0%|          | 0/788721 [00:00<?, ?it/s]

{'prec@10': 0.0165360988579986, 'recall@10': 0.08030196834078777, 'map@10': 0.02468175196165356, 'novelty': 7.313567391305034, 'serendipity': 0.00013973276059021567}

==================== Fold 3
{'End date': Timestamp('2021-08-15 00:00:00', freq='W-SUN'),
 'Start date': Timestamp('2021-08-08 00:00:00', freq='W-SUN'),
 'Test': 276699,
 'Test items': 6715,
 'Test users': 101983,
 'Train': 4587708,
 'Train items': 15404,
 'Train users': 842129}


  0%|          | 0/842129 [00:00<?, ?it/s]

  0%|          | 0/842129 [00:00<?, ?it/s]

  0%|          | 0/842129 [00:00<?, ?it/s]

In [ ]:
# {'prec@10': 0.017548733290595246, 'recall@10': 0.09229051426875087, 'map@10': 0.028046326852875662, 'novelty': 7.218071597702881, 'serendipity': 0.00011776222034225459}
# {'prec@10': 0.0173897662954172, 'recall@10': 0.08785209775497947, 'map@10': 0.027736285607939296, 'novelty': 7.283353827592802, 'serendipity': 0.00013563620249392434}
# {'prec@10': 0.016646972887036482, 'recall@10': 0.08212025031571943, 'map@10': 0.025672975340303293, 'novelty': 7.297639181209256, 'serendipity': 0.00014189979239409036}
# {'prec@10': 0.0165360988579986, 'recall@10': 0.08030196834078777, 'map@10': 0.02468175196165356, 'novelty': 7.313567391305034, 'serendipity': 0.00013973276059021567}
# {'prec@10': 0.011469558651932188, 'recall@10': 0.055158258398103545, 'map@10': 0.01358004273540176, 'novelty': 7.363768234146729, 'serendipity': 0.00011856235493160416}

In [ ]:
df_metrics = pd.DataFrame(results)
df_metrics

,fold,model,prec@10,recall@10,map@10,novelty,serendipity
0,0,blending,0.017549,0.092291,0.028046,7.218072,0.000118
1,1,blending,0.017390,0.087852,0.027736,7.283354,0.000136
2,2,blending,0.016647,0.082120,0.025673,7.297639,0.000142
3,3,blending,0.016536,0.080302,0.024682,7.313567,0.000140
4,4,blending,0.011470,0.055158,0.013580,7.363768,0.000119


In [ ]:
df_metrics.mean()[1:]

prec@10        0.015918
recall@10      0.079545
map@10         0.023943
novelty        7.295280
serendipity    0.000131
dtype: float64

In [ ]:
pd.DataFrame(
    {
        "CosineRecommender": [0.004688, 0.024117, 0.004552, 7.976401, 0.000066],
        "model_with_popular": [0.012763, 0.068503, 0.012446, 7.244911, 0.000063],
        "model_genres_rank": [0.009441, 0.045990, 0.015219, 6.882398, 0.000073],
        "model_genres_rank_with_popular": [
            0.017516,
            0.090376,
            0.023113,
            6.150908,
            0.000069,
        ],
        "blending": [0.015918, 0.079545, 0.023943, 7.295280, 0.000131]
    },
    index=["prec@10", "recall@10", "map@10", "novelty", "serendipity"],
)

,CosineRecommender,model_with_popular,model_genres_rank,model_genres_rank_with_popular,blending
prec@10,0.004688,0.012763,0.009441,0.017516,0.015918
recall@10,0.024117,0.068503,0.045990,0.090376,0.079545
map@10,0.004552,0.012446,0.015219,0.023113,0.023943
novelty,7.976401,7.244911,6.882398,6.150908,7.295280
serendipity,0.000066,0.000063,0.000073,0.000069,0.000131


The result is slightly less prec@10, recall@10, a good map@10, novelty value and a much higher serendipity. It may be more interesting for the user to watch a movie that is unexpected for him.

# Offline validation

In [ ]:
from rectools.metrics import (
    Precision,
    Recall,
    MeanInvUserFreq,
    Serendipity,
    calc_metrics,
    MAP,
)

# calculate several classic (precision@k and recall@k) and "beyond accuracy" metrics
metrics = {
    "prec@10": Precision(k=10),
    "recall@10": Recall(k=10),
    "novelty": MeanInvUserFreq(k=10),
    "serendipity": Serendipity(k=10),
    "map@10": MAP(k=10),
}

# few simple models to compare
models = {
    "tfidf_itemknn_15": TFIDFRecommender(),
    "tfidf_itemknn_30": TFIDFRecommender()
}

4 days size fold 

In [ ]:
# setting for cv 
n_folds = 5
unit = 4
n_units = 1

last_date = interactions[Columns.Datetime].max().normalize()
start_date = last_date - pd.Timedelta((n_folds + 1) * n_units * unit, unit='days')  
print(f"Start date and last date of the test fold: {start_date, last_date}")

Start date and last date of the test fold: (Timestamp('2021-07-29 00:00:00'), Timestamp('2021-08-22 00:00:00'))


In [ ]:
from rectools.model_selection import TimeRangeSplit

periods = n_folds + 1
freq = f"{unit}D"
print(
    f"start_date: {start_date}\n"
    f"last_date: {last_date}\n"
    f"periods: {periods}\n"
    f"freq: {freq}\n"
)
    
date_range = pd.date_range(start=start_date, periods=periods, freq=freq, tz=last_date.tz)
print(f"Test fold borders: {date_range.values.astype('datetime64[D]')}")

# generator of folds
cv = TimeRangeSplit(
    date_range=date_range,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)
print(f"Real number of folds: {cv.get_n_splits(interactions)}")

start_date: 2021-07-29 00:00:00
last_date: 2021-08-22 00:00:00
periods: 6
freq: 4D

Test fold borders: ['2021-07-29' '2021-08-02' '2021-08-06' '2021-08-10' '2021-08-14'
 '2021-08-18']
Real number of folds: 5


In [ ]:
%%time
results = []

fold_iterator = cv.split(interactions, collect_fold_stats=True)

for i_fold, (train_ids, test_ids, fold_info) in enumerate(fold_iterator):
    print(f"\n==================== Fold {i_fold}")
    pprint(fold_info)

    df_train = interactions.iloc[train_ids].copy()
    df_test = interactions.iloc[test_ids][Columns.UserItem].copy()

    catalog = df_train[Columns.Item].unique()
    n=0

    for model_name, model in models.items():
        n += 15
        userknn_model = UserKnn(model=model, N_users=n)
        userknn_model.fit(df_train)
   
        recos = userknn_model.predict(df_test)
        recos = get_genre_rank(recos, items, user_genres)
        extra_recos = get_popular_recos(recos, popular)
        recos = recos.append(extra_recos)
       
        metric_values = calc_metrics(
            metrics,
            reco=recos,
            interactions=df_test,
            prev_interactions=df_train,
            catalog=catalog,
        )
    
        fold = {"fold": i_fold, "model": model_name}
        fold.update(metric_values)
        results.append(fold)


==================== Fold 0
{'End date': Timestamp('2021-08-02 00:00:00', freq='4D'),
 'Start date': Timestamp('2021-07-29 00:00:00', freq='4D'),
 'Test': 161159,
 'Test items': 5905,
 'Test users': 70871,
 'Train': 4040680,
 'Train items': 15145,
 'Train users': 764303}


  0%|          | 0/764303 [00:00<?, ?it/s]

  0%|          | 0/764303 [00:00<?, ?it/s]


==================== Fold 1
{'End date': Timestamp('2021-08-06 00:00:00', freq='4D'),
 'Start date': Timestamp('2021-08-02 00:00:00', freq='4D'),
 'Test': 152688,
 'Test items': 5669,
 'Test users': 67737,
 'Train': 4266013,
 'Train items': 15237,
 'Train users': 797423}


  0%|          | 0/797423 [00:00<?, ?it/s]

...


==================== Fold 2
{'End date': Timestamp('2021-08-10 00:00:00', freq='4D'),
 'Start date': Timestamp('2021-08-06 00:00:00', freq='4D'),
 'Test': 167692,
 'Test items': 5854,
 'Test users': 72742,
 'Train': 4474647,
 'Train items': 15356,
 'Train users': 826429}


  0%|          | 0/826429 [00:00<?, ?it/s]

  0%|          | 0/826429 [00:00<?, ?it/s]


==================== Fold 3
{'End date': Timestamp('2021-08-14 00:00:00', freq='4D'),
 'Start date': Timestamp('2021-08-10 00:00:00', freq='4D'),
 'Test': 164016,
 'Test items': 5899,
 'Test users': 71612,
 'Train': 4701956,
 'Train items': 15444,
 'Train users': 857700}


  0%|          | 0/857700 [00:00<?, ?it/s]

In [ ]:
df_metrics = pd.DataFrame(results)

In [ ]:
df_metrics

In [ ]:
df_metrics.to_csv("cv_4.csv",index=False)

14 days size fold 

In [ ]:
# setting for cv 
n_folds = 5
unit = 14
n_units = 1

last_date = interactions[Columns.Datetime].max().normalize()
start_date = last_date - pd.Timedelta((n_folds + 1) * n_units * unit, unit='days')  
print(f"Start date and last date of the test fold: {start_date, last_date}")

Start date and last date of the test fold: (Timestamp('2021-05-30 00:00:00'), Timestamp('2021-08-22 00:00:00'))


In [ ]:
from rectools.model_selection import TimeRangeSplit

periods = n_folds + 1
freq = f"{unit}D"
print(
    f"start_date: {start_date}\n"
    f"last_date: {last_date}\n"
    f"periods: {periods}\n"
    f"freq: {freq}\n"
)
    
date_range = pd.date_range(start=start_date, periods=periods, freq=freq, tz=last_date.tz)
print(f"Test fold borders: {date_range.values.astype('datetime64[D]')}")

# generator of folds
cv = TimeRangeSplit(
    date_range=date_range,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)
print(f"Real number of folds: {cv.get_n_splits(interactions)}")

start_date: 2021-05-30 00:00:00
last_date: 2021-08-22 00:00:00
periods: 6
freq: 14D

Test fold borders: ['2021-05-30' '2021-06-13' '2021-06-27' '2021-07-11' '2021-07-25'
 '2021-08-08']
Real number of folds: 5


In [ ]:
%%time
results = []

fold_iterator = cv.split(interactions, collect_fold_stats=True)

for i_fold, (train_ids, test_ids, fold_info) in enumerate(fold_iterator):
    print(f"\n==================== Fold {i_fold}")
    pprint(fold_info)

    df_train = interactions.iloc[train_ids].copy()
    df_test = interactions.iloc[test_ids][Columns.UserItem].copy()

    catalog = df_train[Columns.Item].unique()
    n=0

    for model_name, model in models.items():
        n += 15
        userknn_model = UserKnn(model=model, N_users=n)
        userknn_model.fit(df_train)
   
        recos = userknn_model.predict(df_test)
        recos = get_genre_rank(recos, items, user_genres)
        extra_recos = recos.groupby("user_id").agg({"rank": np.max})
        extra_recos = extra_recos.loc[extra_recos["rank"] < 10].reset_index()
        extra_recos["add_films"] = 10 - extra_recos["rank"]
        extra_recos = get_popular_recos(extra_recos, recos, popular)
        recos = recos.append(extra_recos)
       
        metric_values = calc_metrics(
            metrics,
            reco=recos,
            interactions=df_test,
            prev_interactions=df_train,
            catalog=catalog,
        )
    
        fold = {"fold": i_fold, "model": model_name}
        fold.update(metric_values)
        results.append(fold)


==================== Fold 0
{'End date': Timestamp('2021-06-13 00:00:00', freq='14D'),
 'Start date': Timestamp('2021-05-30 00:00:00', freq='14D'),
 'Test': 216792,
 'Test items': 6530,
 'Test users': 69914,
 'Train': 1480359,
 'Train items': 13054,
 'Train users': 334848}


  0%|          | 0/334848 [00:00<?, ?it/s]

  0%|          | 0/334848 [00:00<?, ?it/s]


==================== Fold 1
{'End date': Timestamp('2021-06-27 00:00:00', freq='14D'),
 'Start date': Timestamp('2021-06-13 00:00:00', freq='14D'),
 'Test': 346548,
 'Test items': 6739,
 'Test users': 108962,
 'Train': 1937299,
 'Train items': 13611,
 'Train users': 431674}


  0%|          | 0/431674 [00:00<?, ?it/s]

...

  0%|          | 0/640144 [00:00<?, ?it/s]


==================== Fold 4
{'End date': Timestamp('2021-08-08 00:00:00', freq='14D'),
 'Start date': Timestamp('2021-07-25 00:00:00', freq='14D'),
 'Test': 451973,
 'Test items': 7736,
 'Test users': 133735,
 'Train': 3838180,
 'Train items': 15061,
 'Train users': 734701}


  0%|          | 0/734701 [00:00<?, ?it/s]

  0%|          | 0/734701 [00:00<?, ?it/s]

CPU times: user 2h 57min 31s, sys: 1min 28s, total: 2h 59min
Wall time: 2h 10min 46s


In [ ]:
df_metrics = pd.DataFrame(results)

In [ ]:
df_metrics

,fold,model,prec@10,recall@10,map@10,novelty,serendipity
0,0,tfidf_itemknn_15,0.013355,0.061950,0.018226,8.707979,0.000078
1,0,tfidf_itemknn_30,0.013913,0.063537,0.019285,8.475191,0.000088
2,1,tfidf_itemknn_15,0.038392,0.179524,0.046083,7.192096,0.000305
3,1,tfidf_itemknn_30,0.036666,0.171454,0.044908,7.161658,0.000272
4,2,tfidf_itemknn_15,0.040291,0.195735,0.050086,6.247142,0.000065
5,2,tfidf_itemknn_30,0.039604,0.192157,0.050630,6.320945,0.000069
6,3,tfidf_itemknn_15,0.024847,0.112071,0.032085,6.138774,0.000071
7,3,tfidf_itemknn_30,0.024955,0.112395,0.033562,6.216366,0.000079
8,4,tfidf_itemknn_15,0.021142,0.090492,0.023365,6.197153,0.000078
9,4,tfidf_itemknn_30,0.021098,0.089845,0.023706,6.281434,0.000087


In [ ]:
df_metrics.to_csv("cv_14.csv",index=False)

In [ ]:
df_metrics.groupby('model').mean()

,fold,prec,recall,map,nov,ser
model,,,,,,
tfidf_itemknn_15,2.0,0.027605,0.127954,0.033969,6.896629,0.000119
tfidf_itemknn_30,2.0,0.027247,0.125878,0.034418,6.891119,0.000119
